# Dependencies

In [ ]:
# global dependencies
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp

In [ ]:
# local dependencies
from utilities.filters import gaussian as gaussian_filter

In [ ]:
# to stop printing the last returned value in each cell to the output
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "none"

# fft instead of circular convolution

In [ ]:
# load cameraman.tif
cm = plt.imread('./resources/CH02_Fig0222(b)(cameraman).tif')

# create a gaussian noise
gaussian_noise = np.random.normal(loc= 0, scale= 10, size= cm.shape)

# apply gaussian noise to the image
noisy_cm = (cm + gaussian_noise).clip(0, 255).astype(np.uint8)

# plot
fig, axs = plt.subplots(nrows= 1, ncols= 2, figsize= (8, 4), layout= 'compressed')

axs[0].imshow(cm, cmap= 'gray')
axs[0].set_title('Original image')
axs[1].imshow(noisy_cm, cmap= 'gray')
axs[1].set_title('Noisy image')

for ax in fig.axes:
    ax.axis('off')

plt.show()

In [ ]:
# smooth in spatial domain [using convolution]
glpf = gaussian_filter(size= (5, 5), sigma= 1, norm= True)

# convolution
denoised_cm_1 = sp.signal.convolve2d(noisy_cm, glpf, mode= 'same', boundary= 'fill')

# clip and dtype
denoised_cm_1 = denoised_cm_1.clip(0, 255).astype(np.uint8)

# plot
fig, axs = plt.subplots(nrows= 1, ncols= 3, figsize= (16, 8), layout= 'compressed')

axs[0].imshow(cm, cmap= 'gray')
axs[0].set_title('Original image')
axs[1].imshow(noisy_cm, cmap= 'gray')
axs[1].set_title('Noisy image')
axs[2].imshow(denoised_cm_1, cmap= 'gray')
axs[2].set_title('Denoised image')

for ax in fig.axes:
    ax.axis('off')

plt.show()

In [ ]:
# smooth in frequency domain [using fft instead of circular convolution]

# image size : MxM, filter size : NxN -> upscale to (M+N-1, M+N-1)
size = (cm.shape[0] + glpf.shape[1] - 1, cm.shape[0] + glpf.shape[1] - 1)

# pad noisy image to size M+N-1
padded_noisy_cm = np.zeros(shape= size)
padded_noisy_cm[:cm.shape[0], :cm.shape[1]] = noisy_cm

# pad filter to size M+N-1
padded_glpf = np.zeros(shape= size)
padded_glpf[:glpf.shape[0], :glpf.shape[1]] = glpf

# plot
fig, axs = plt.subplots(nrows= 1, ncols= 2, figsize= (8, 4), layout= 'compressed')

axs[0].imshow(padded_noisy_cm, cmap= 'gray')
axs[0].set_title('padded noisy image')
axs[1].imshow(padded_glpf, cmap= 'gray')
axs[1].set_title('padded filter')
axs[1].add_artist(plt.arrow(40, 40, -25, -25, color='red', head_width=10, head_length=10))

for ax in fig.axes:
    ax.axis('off')

plt.show()

In [ ]:
# fft
fft_padded_noisy_cm = np.fft.fftshift(np.fft.fft2(padded_noisy_cm))
abs_fft_padded_noisy_cm = np.abs(fft_padded_noisy_cm)

fft_padded_glpf = np.fft.fftshift(np.fft.fft2(padded_glpf))
abs_fft_padded_glpf = np.abs(fft_padded_glpf)

fft_multiplication = np.multiply(fft_padded_noisy_cm, fft_padded_glpf)
abs_fft_multiplication = np.abs(fft_multiplication)

# plot
fig, axs = plt.subplots(nrows= 1, ncols= 3, figsize= (16, 8), layout= 'compressed')

axs[0].imshow(np.log2(abs_fft_padded_noisy_cm + 1), cmap= 'gray')
axs[0].set_title('Magnitude(noisy_cm)')
axs[1].imshow(np.log2(abs_fft_padded_glpf + 1), cmap= 'gray')
axs[1].set_title('Magnitude(filter)')
axs[2].imshow(np.log2(abs_fft_multiplication + 1), cmap= 'gray')
axs[2].set_title('Magnitude(noisy_cm * filter)')

for ax in fig.axes:
    ax.axis('off')

plt.show()

In [ ]:
# reconstruction
ifft_noisy_cm = np.fft.ifft2(np.fft.ifftshift(fft_multiplication)).real

# downscale to size: cm.shape
ifft_noisy_cm = ifft_noisy_cm[glpf.shape[0] // 2:cm.shape[0] + glpf.shape[0] // 2, glpf.shape[1] // 2:cm.shape[1] + glpf.shape[1] // 2]

# clip & dtype
denoised_cm_2 = ifft_noisy_cm.clip(0, 255).astype(np.uint8)

# plot
fig, axs = plt.subplots(nrows= 1, ncols= 3, figsize= (16, 8), layout= 'compressed')

axs[0].imshow(noisy_cm, cmap= 'gray')
axs[0].set_title('Noisy image')
axs[1].imshow(denoised_cm_1, cmap= 'gray')
axs[1].set_title('Denoised [convolution]')
axs[2].imshow(denoised_cm_2, cmap= 'gray')
axs[2].set_title('Denoised [fft]')

for ax in fig.axes:
    ax.axis('off')

plt.show()